<a href="https://colab.research.google.com/github/AvantiShri/dueling_init/blob/master/DuelingInit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!pip install simdna
!pip install keras-genomics

     |████████████████████████████████| 634kB 8.1MB/s 
  ERROR: Failed building wheel for simdna
  Running setup.py clean for simdna
Failed to build simdna
    Running setup.py install for simdna ... done
  Created wheel for keras-genomics: filename=keras_genomics-0.1.2.1-cp36-none-any.whl size=10699 sha256=f4ae03ad5263da73b95c2242eb7b9a6198ce7e905a6d808450dc4c7c726a0f39
  Stored in directory: /root/.cache/pip/wheels/2a/8e/eb/17f924ed090e1af0773d925dadd42339639e1b87c8ff17dff8
Successfully built keras-genomics


In [10]:
!rm *.fa *.simdata *.txt
!densityMotifSimulation.py --prefix pos --motifNames GATA_disc1 --mean-motifs 1 --min-motifs 1 --max-motifs 3 --rc-prob 0.5 --zero-prob 0.2 --numSeqs 1000 --seqLength 400 --seed 1234
!emptyBackground.py --prefix neg --seqLength 400 --numSeqs 1000

from glob import glob
import numpy as np
import sklearn
import sklearn.model_selection
import simdna
from simdna import synthetic

ltr = {
    'A': [1,0,0,0], 'C': [0,1,0,0], 'G': [0,0,1,0], 'T': [0,0,0,1]
}

def onehot_encode(seqs):
  return np.array([ [ltr[x] for x in seq] for seq in seqs])

def read_fasta(file):
  return [x.rstrip() for i,x in enumerate(open(file)) if i%2==1]

pos_seqs = read_fasta(glob("DensityEmbedding_prefix-pos*.fa")[0])
neg_seqs = read_fasta(glob("EmptyBackground_prefix-neg*.fa")[0])

#Replace the revcomp instances
rng = np.random.RandomState(1234)
def get_random_string(length):
  return rng.choice(np.array(['A','C','G','T']),
             p=np.array([0.3, 0.2, 0.2, 0.3]),
             size=length)
pos_embeddings = synthetic.core.read_simdata_file(
    glob("DensityEmbedding_prefix-pos*.simdata")[0]).embeddings
rcreplaced_pos_seqs = []
for seq, embeddings in zip(pos_seqs, pos_embeddings):
  new_seq = list(seq)
  for embedding in embeddings:
    if "revComp" in embedding.what.stringDescription:
      new_seq[embedding.startPos:embedding.startPos+len(embedding.what)] =\
        get_random_string(len(embedding.what))
  rcreplaced_pos_seqs.append("".join(new_seq))

#all_onehot = onehot_encode(pos_seqs+neg_seqs)
all_onehot = onehot_encode(rcreplaced_pos_seqs+neg_seqs)
all_labels = np.array([[1] for x in pos_seqs]+[[0] for x in neg_seqs])

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    all_onehot, all_labels, test_size=0.2, random_state=1234)

In [13]:
import tensorflow as tf
import numpy as np
import keras
from keras_genomics.layers.convolutional import RevCompConv1D
from sklearn.metrics import roc_auc_score


class RevCompSumPool(keras.engine.Layer):
    def __init__(self, **kwargs): 
        super(RevCompSumPool, self).__init__(**kwargs)

    def build(self, input_shape):
        self.num_input_chan = input_shape[2]
        super(RevCompSumPool, self).build(input_shape)

    def call(self, inputs): 
        #divide by sqrt 2 for variance preservation
        inputs = (inputs[:,:,:int(self.num_input_chan/2)] +
                  inputs[:,:,int(self.num_input_chan/2):][:,::-1,::-1])/(1.41421356237)
        return inputs
      
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], int(input_shape[2]/2))


def set_seed(seed):
  np.random.seed(seed)
  tf.set_random_seed(seed)


def make_standard_model(filter_width, filt_per_layer, num_layers):
  model = keras.models.Sequential()
  model.add(keras.layers.Conv1D(filters=filt_per_layer,
                                kernel_size=filter_width,
                                input_shape=X_train.shape[1:],
                                activation="relu",
                                kernel_initializer="he_normal"))
  for i in range(1,num_layers):
    model.add(keras.layers.Conv1D(filters=filt_per_layer,
                                kernel_size=filter_width,
                                activation="relu",
                                kernel_initializer="he_normal"))
  model.add(keras.layers.GlobalAveragePooling1D())
  model.add(keras.layers.Dense(1, activation="sigmoid"))
  return model


def make_rcconv_model(filter_width, filt_per_layer, num_layers):
  model = keras.models.Sequential()
  model.add(RevCompConv1D(filters=filt_per_layer,
                                       kernel_size=filter_width,
                                       input_shape=X_train.shape[1:],
                                       activation="relu",
                                       kernel_initializer="he_normal"))
  for i in range(1,num_layers):
    model.add(RevCompConv1D(filters=filt_per_layer,
                                         kernel_size=filter_width,
                                         activation="relu",
                                         kernel_initializer="he_normal"))
  model.add(keras.layers.GlobalAveragePooling1D())
  model.add(keras.layers.Dense(1, activation="sigmoid"))
  return model


def make_rcfull_model(filter_width, filt_per_layer, num_layers):
  model = keras.models.Sequential()
  model.add(RevCompConv1D(filters=filt_per_layer,
                                       kernel_size=filter_width,
                                       input_shape=X_train.shape[1:],
                                       activation="relu",
                                       kernel_initializer="he_normal"))
  for i in range(1,num_layers):
    model.add(RevCompConv1D(filters=filt_per_layer,
                                         kernel_size=filter_width,
                                         activation="relu",
                                         kernel_initializer="he_normal"))
  model.add(RevCompSumPool())
  model.add(keras.layers.GlobalAveragePooling1D())
  model.add(keras.layers.Dense(1, activation="sigmoid"))
  return model


def train_model(modelcreator, seed):
  set_seed(seed)
  model = modelcreator()
  early_stopping_callback = keras.callbacks.EarlyStopping(
                              patience=10, restore_best_weights=True)
  model.compile(optimizer="adam", loss="binary_crossentropy",
                metrics=["accuracy"])
  history = model.fit(X_train, y_train,
            validation_data=(X_test, y_test), epochs=100,
            callbacks=[early_stopping_callback])
  model.set_weights(early_stopping_callback.best_weights)
  y_test_preds = model.predict(X_test)
  return roc_auc_score(y_true=y_test, y_score=y_test_preds)


def train_many_models(modelcreator, num_seeds):
  aucs = []
  for seednum in range(num_seeds):
    seed = seednum*100
    print("Seed:",seed)
    auc = train_model(modelcreator, seed)
    aucs.append(auc)
    print("AUC:",auc)
  print("AUCS:",aucs)
  return aucs

In [14]:
archname_and_factory = [    
#('rcconv_width15_10filt_3layer', lambda: make_rcconv_model(filter_width=15, filt_per_layer=20, num_layers=3)),
#('rcfull_width15_10filt_3layer', lambda: make_rcfull_model(filter_width=15, filt_per_layer=20, num_layers=3)),
('standard_width15_10filt_3layer', lambda: make_standard_model(filter_width=15, filt_per_layer=20, num_layers=3)),
('standard_width7_10filt_3layer', lambda: make_standard_model(filter_width=7, filt_per_layer=20, num_layers=3)),
]

archname_to_aucs = {}
for archname, factory in archname_and_factory:
  print(archname)
  aucs = train_many_models(factory, num_seeds=5)
  archname_to_aucs[archname] = aucs


standard_width15_10filt_3layer
Seed: 0
Train on 1600 samples, validate on 400 samples
Epoch 1/100
1600/1600 [==============================] - 1s 888us/step - loss: 0.6993 - accuracy: 0.4963 - val_loss: 0.6934 - val_accuracy: 0.5050
Epoch 2/100
1600/1600 [==============================] - 0s 213us/step - loss: 0.6948 - accuracy: 0.5025 - val_loss: 0.6931 - val_accuracy: 0.5050
Epoch 3/100
1600/1600 [==============================] - 0s 200us/step - loss: 0.6937 - accuracy: 0.4869 - val_loss: 0.6931 - val_accuracy: 0.5050
Epoch 4/100
1600/1600 [==============================] - 0s 211us/step - loss: 0.6933 - accuracy: 0.4981 - val_loss: 0.6930 - val_accuracy: 0.5050
Epoch 5/100
1600/1600 [==============================] - 0s 218us/step - loss: 0.6930 - accuracy: 0.5088 - val_loss: 0.6933 - val_accuracy: 0.4950
Epoch 6/100
1600/1600 [==============================] - 0s 256us/step - loss: 0.6927 - accuracy: 0.5181 - val_loss: 0.6932 - val_accuracy: 0.4950
Epoch 7/100
1600/1600 [=========

In [15]:
for archname, aucs in archname_to_aucs.items():
  print(archname, aucs)

standard_width15_10filt_3layer [0.6691919191919191, 0.6280878087808781, 0.4923742374237424, 0.6833183318331834, 0.4877987798779878]
standard_width7_10filt_3layer [0.6977697769776977, 0.663991399139914, 0.6722172217221722, 0.6900940094009401, 0.6847184718471848]
